In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y  
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121 
!pip install unsloth 
# pip install --upgrade typing-extensions 

In [ ]:
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
import json

In [ ]:
import gc
torch.cuda.empty_cache()
# del(model)
gc.collect()

In [ ]:


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!



In [ ]:
import os
from transformers import TextStreamer

# Persistent messages list
messages = []

# Initialize TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
while True:
    # Add user input to the conversationH
    user_input = input("You: ")
    messages.append({"role": "user", "content": user_input})
    
    # Tokenize and prepare input for the model
    chat_history = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(chat_history, return_tensors='pt').to("cuda")
    
    # Generate response
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        streamer=None,  # Set to None for now; replace with `text_streamer` if desired
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
        temperature = 0.8
    )
    
    # Decode the entire output and slice to get the generated response
    generated_tokens = output[0][inputs.input_ids.shape[1]:]  # Exclude input tokens
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    print(f"Assistant: {response}")
    
    # Add assistant response to the conversation
    messages.append({"role": "assistant", "content": response})